In [24]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [25]:
import os
from pinecone import Pinecone as pcn
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from IPython.display import Markdown
import gradio as gr
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI 

### 01 Get API Keys

In [26]:
api_key = os.environ.get("PINECONE_API_KEY")
openai_api_key = os.getenv('OPENAI_API_KEY')

In [27]:
# configure client
pc = pcn(api_key=api_key)
index_name = 'recipes-index'
index = pc.Index(index_name)
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1106}},
 'total_vector_count': 1106}

### 02 Set Embedding Model

In [28]:
embed_model = "text-embedding-3-small"
embed = OpenAIEmbeddings(model=embed_model)

In [29]:
text_field = "description"

vectorStore = PineconeVectorStore(
  index, embed, text_field
)

In [38]:
def get_recipe(query, history=None):# completion llm  
    llm = ChatOpenAI(  
        model_name='gpt-4o-mini',  
        temperature=0.0  
    ) 

    retriever = vectorStore.similarity_search_with_score(query)
    # we can set a similarity score threshold and only return documents with a score above that threshold.
    # search_kwargs={"score_threshold": 0.5}
    # We can also limit the number of documents k returned by the retriever.
    # retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

    system_prompt = (    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
    )

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    response = rag_chain.invoke({"input": query})
    return response["answer"]

## Gradio App

In [39]:
demo = gr.ChatInterface(
  get_recipe,
  title="Recipes Generator",
  description="Ask recipes recommendations based on ingredients and instructions",
  examples=["Recommend a recipe with chicken", "A low-calorie chicken recipe", "What can I do with brocolli"],
)

demo.launch()

Running on local URL:  http://127.0.0.1:7874

Thanks for being a Gradio user! If you have questions or feedback, please join our Discord server and chat with us: https://discord.gg/feTf9x3ZSB

To create a public link, set `share=True` in `launch()`.


### retriever [(Document(id='982', metadata={'calories': '502', 'carbs': '9g', 'cook_time': '40 mins', 'diet_type': 'Egg-free', 'difficulty': 'Easy', 'fat': '30g', 'fibre': '1g', 'id': '982', 'ingredients': '4chicken breasts, skin left on, 2-3 tbspolive or rapeseed oil, 1onion, finely chopped, 3garlic cloves, crushed or finely grated, 1 tbspdried mixed herbs, 1 tbspplain flour, 200mlwhite wine, 250mlchicken stock, 125mldouble cream, choppedparsley, to serve (optional)', 'instructions': 'Season the chicken all over with salt and freshly ground black pepper. Heat 2 tbsp of the oil in a large lidded frying pan over a medium heat and fry the chicken, skin-side down, for 8-10 mins until the skin is golden. Turn the chicken over and cook for 6-8 mins more until golden all over. Remove to a plate and set aside. If the pan is dry, drizzle in another 1 tbsp oil, then reduce the heat to medium-low and fry the onions for 8-10 mins until softened but not golden. Stir in the garlic and cook for 1 m

Traceback (most recent call last):
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 536, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1935, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1518, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/acrisvall/recipes_rag/.venv/lib/python3.12/site-packages/gradio/utils.py", line 793, in async_wrapper
    